In [0]:
import pandas as pd
import datetime 


In [0]:
# USA

df_USA = pd.read_excel("/content/drive/My Drive/野村_文字探勘_加減碼_組別一/美國.xlsx", sheet_name= "美林時鐘")
df_USA['PPI-CPI'] = 0

for i in range(len(df_USA)):
  df_USA.loc[i,'PPI-CPI'] =  df_USA.loc[i,'PPI'] - df_USA.loc[i,'CPI'] 

df_USA

,Date,CPI,PPI,PMI,M1,M2,CPI_nofood,Import price growth,Export price growth,PPI-CPI
0,2020-03-01,0.015,0.007,48.5,0.009169,-0.026797,0.000165,-0.023331,-0.016077,-0.008
1,2020-02-01,0.023,0.013,49.2,-0.015994,-0.000135,0.004752,-0.007188,-0.011129,-0.010
2,2020-01-01,0.025,0.021,50.7,0.000127,-0.002152,0.004035,0.001600,0.006400,-0.004
3,2019-12-01,0.023,0.013,50.8,0.004277,-0.000633,-0.000653,0.002406,-0.001597,-0.010
4,2019-11-01,0.021,0.011,51.9,-0.006907,-0.002003,0.000185,0.001606,0.000799,-0.010
...,...,...,...,...,...,...,...,...,...,...
94,2012-05-01,0.017,0.007,50.8,-0.002382,-0.000661,0.001304,-0.014573,-0.005197,-0.010
95,2012-04-01,0.023,0.019,50.8,-0.003600,-0.000858,0.002483,-0.000693,0.004474,-0.004
96,2012-03-01,0.027,0.023,51.5,0.006866,0.001652,0.003818,0.014065,0.007513,-0.004
97,2012-02-01,0.029,0.028,51.4,0.006599,0.000561,0.002764,0.000000,0.004528,-0.001


In [0]:
#======= PPI - CPI =======
def PPI_CPI( n, n_3): # 第 n 期 ppi - cpi 與第 n-3期 ppi - cpi
   if n<0 and n_3 <0 and (n - n_3) < 0: return 3
   if n - n_3 >0: return 2 # ppi - cpi 擴張 => 復甦
   elif n - n_3 <0:  # ppi - cpi 縮小 => 擴張 or 滯脹
    if n < 0: return 4  # 最新的 ppi 為負數 => 滯脹 
    else: return 1

#======= PMI =======
def PMI(pmi, pmi_3):
  if pmi - pmi_3 >0 and pmi > 50 : return 1
  elif pmi - pmi_3 <0 and pmi<= 52: return 4
  elif pmi - pmi_3 >0 and pmi_3 < 50: return 2
  else: return 3

#======= M1 & M2 =======
def M1_M2(m1, m1_3, m2, m2_3):  # M1 是貨幣寬鬆狀況，M2 是信用寬鬆狀況
  if m1 - m1_3 > 0:  # 寬鬆貨幣 => 復甦 or 衰退
    if m2 - m2_3 >0: return 2
    else: return 3 
  else:  # 擴張 or 滯脹
    if m2 - m2_3 >0: return 1  # 緊縮貨幣 + 仍寬信用
    else: return 4

#======= CPI_nofood =======
def CPI_nofood(cf, cf_3, ppi, ppi_3):
  if cf < 0 and cf_3 >= 0: return 2    # 原本下跌後停止下跌
  if ppi > 0 and ppi_3 >0 and cf > 0: return 1
  if ppi<0 and ppi_3 <0 and (cf - cf_3)>=0: return 4
  else: return 3
   
#======= Import Price Index =======
def Import(n, n_3):
  if n >0 and n_3 >0 and n <= n_3: return 1   # 進口增長率放緩
  if n< 0 and n_3 < 0 :return 4  # 進口增長率下降趨勢
  if n <= 0 : return 3
  else: return 2

#======= Determine =======
def determine_stata(l):
 count = [0,0,0,0]
  
 for i in range(len(l)):
    if l[i] == 1: count[0] += 1
    elif l[i] == 2: count[1] += 1
    elif l[i] == 3: count[2] += 1
    elif l[i] == 4: count[3] += 1
  
    try:
      return (count.index(max(count)) + 1)
    except:
      return None


In [0]:
# 一次看一季

indexes = [0,0,0,0,0]

count_1 = 0
count_2 = 0
count_3 = 0
count_4 = 0
df_USA['state'] = ""

for i in range(len(df_USA)-4):
  indexes[0] = PPI_CPI(df_USA.loc[i,'PPI-CPI'], df_USA.loc[i+3,'PPI-CPI'])
  indexes[1] = PMI(df_USA.loc[i,'PMI'], df_USA.loc[i+3,'PMI'])
  indexes[2] = M1_M2(df_USA.loc[i,'M1'], df_USA.loc[i+3,'M1'], df_USA.loc[i,'M2'], df_USA.loc[i+3,'M2'])
  indexes[3] = CPI_nofood(df_USA.loc[i,'CPI_nofood'], df_USA.loc[i+3,'CPI_nofood'], df_USA.loc[i,'PPI'], df_USA.loc[i+3,'PPI'])
  indexes[4] = PMI(df_USA.loc[i,'Import price growth'], df_USA.loc[i+3,'Import price growth'])

  df_USA.loc[i,'state'] = determine_stata(indexes)


In [0]:
pd.set_option('display.max_rows', None)
df_USA = df_USA.sort_values(by='Date', ignore_index= True)
df_USA

,Date,CPI,PPI,PMI,M1,M2,CPI_nofood,Import price growth,Export price growth,PPI-CPI,state
0,2012-01-01,0.029,0.031,51.3,-0.000546,-0.000624,0.002192,0.000000,0.003028,0.002,
1,2012-02-01,0.029,0.028,51.4,0.006599,0.000561,0.002764,0.000000,0.004528,-0.001,
2,2012-03-01,0.027,0.023,51.5,0.006866,0.001652,0.003818,0.014065,0.007513,-0.004,
3,2012-04-01,0.023,0.019,50.8,-0.003600,-0.000858,0.002483,-0.000693,0.004474,-0.004,
4,2012-05-01,0.017,0.007,50.8,-0.002382,-0.000661,0.001304,-0.014573,-0.005197,-0.010,3
5,2012-06-01,0.017,0.007,50.5,-0.006942,0.000418,0.001206,-0.023239,-0.017164,-0.010,3
6,2012-07-01,0.014,0.005,50.7,-0.009112,0.000257,0.000061,-0.007210,0.003797,-0.009,3
7,2012-08-01,0.017,0.020,51.4,-0.003489,0.000451,0.001318,0.012346,0.009077,0.003,2
8,2012-09-01,0.020,0.021,51.3,-0.010726,0.000752,0.002537,0.010043,0.008246,0.001,2
9,2012-10-01,0.022,0.023,52.9,-0.000083,-0.002325,0.002149,0.002841,0.000743,0.001,2
